<a href="https://colab.research.google.com/github/MaralAminpour/ML-BME-Course-UofA-Fall-2023/blob/main/Week-3-Classification-models/4.1.Univariate_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Univariate Logistic Regression

In this notebook we will review the main concepts of linear binary classification using an example of logistic regression. We will look at the univariate case - prediction of **heart failure** from a single feature, **Ejection Fraction**.

## Prepare dataset

The code below
* loads all the important libraries
* implements function `accuracyCV` to calculate classification accuracy using cross-validation
* implements function `plotData` to plot the dataset
* creates feature matrix `X` that contains **Ejection Fraction**
* creates label vector `y` that contains labels **Healthy=0** and **Heart Failure=1**

Run this code.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Calculate cross-validated accuracy
def accuracyCV(model, X, y):
    scores = cross_val_score(model, X, y, cv=5, scoring="accuracy")
    print('Mean cross-validated accuracy: ',round(scores.mean(),2))

# Plot data
def plotData(X,y,adjust_axis = True):
    plt.figure(figsize=(7,3))
    plt.plot(X[y==0], y[y==0], "bo", alpha=0.75, markeredgecolor='k', label = 'Healthy')
    plt.plot(X[y==1], y[y==1], "rd", alpha=0.75, markeredgecolor='k', label = 'Heart Failure')
    plt.plot(np.linspace(-2.5,2,100),np.zeros(100),'k:',alpha=0.5)

    plt.xlabel('Feature: Scaled Ejection Fraction')
    plt.ylabel('Label: Healthy=0, Heart Failure=1')
    plt.legend()

    if adjust_axis:
        plt.axis([-2.5,2,-0.1,1.1])

# This code will download the required data files from GitHub
import requests
def download_data(source, dest):
    base_url = 'https://raw.githubusercontent.com/'
    owner = 'SirTurtle'
    repo = 'ML-BME-UofA-data'
    branch = 'main'
    url = '{}/{}/{}/{}/{}'.format(base_url, owner, repo, branch, source)
    r = requests.get(url)
    f = open(dest, 'wb')
    f.write(r.content)
    f.close()

# Create the temp directory, if it doesn't already exist
import os
if not os.path.exists('temp'):
   os.makedirs('temp')

# Download the data
download_data('Week-3-Classification-models/data/heart_failure_data.csv', 'temp/heart_failure_data.csv')

#Load dataset
df = pd.read_csv('temp/heart_failure_data.csv')
print('Dataframe columns: ', df.keys())
scaler = StandardScaler()
data = df.to_numpy()
X = scaler.fit_transform(data[:,0].reshape(-1,1)) # Ejection Fraction
print('Feature matrix X dimensions: ', X.shape)
y = data[:,2] # Heart Failure
print('Target vector y dimensions: ', y.shape)
plotData(X,y)

A Refresher on What We've Learned So Far on linear classifier and Logostic regression.

**Linear binary classification**

Linear binary classification is a type of classification algorithm that is used to separate data into two classes (hence "binary") based on a linear combination of the features of the data (hence "linear")

**Linear decision function**  

The linear decision function is a mathematical equation that helps in distinguishing between two classes based on a set of features or inputs.

$$
f(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + b
$$

or

$$
f(\mathbf{x}) = w_1 x_1 + w_2 x_2 + \ldots + w_n x_n + b
$$

We have the feature vector, $\mathbf{x}$, and the weight vector, $\mathbf{w}$, defined as:

$$
\mathbf{x} = (1, x_1, x_2, \ldots, x_N)^T
$$

$$
\mathbf{w} = (w_0, w_1, w_2, \ldots, w_N)^T
$$

**Decision boundary**

Decision boundary is typically where the decision function equals zero:

$$ f(\mathbf{x}) = 0 $$.

**Classifying the data points**:

Depending on the sign of $f(\mathbf{x})$, the data point $\mathbf{x}$ is classified into one of the two classes. Typically:
   - If $f(\mathbf{x}) > 0$, we predict the label as 1.
   - If $f(\mathbf{x}) \leq 0$, we predict the label as 0.

**Logistic regression calssifier**

An advantage** of the logistic regression classifier is that the **output of the model is a probability.

This function squashes the output of decision function into rage [0,1].

Sigmoid Function**

The output of the decision function is then transformed using the sigmoid function to yield a probability value. The sigmoid function $( \sigma $) is defined as:


$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

<img src="https://raw.githubusercontent.com/MaralAminpour/ML-BME-Course-UofA-Fall-2023/main/Week-3-Classification-models/imgs/LogisticRegression-fig1.png" width = "300" style="float: right;">


So, applying this to our decision function, we get:

$$
P(y=1|\mathbf{x}) = \sigma(f(\mathbf{x})) = \sigma(\mathbf{w}^T \mathbf{x} + b) = \frac{1}{1 + e^{-(\mathbf{w}^T \mathbf{x} + b)}}
$$

**Probabilities of Each Class**

Using the sigmoid function, we can determine the probability that a given input ($ \mathbf{x} $) belongs to class 1 ($ y = 1 $) or class 0 ($ y = 0 $):

- **Probability of $y = 1 $ given $ \mathbf{x} $**
  
  We already derived this above:

  $$
  P(y=1|\mathbf{x}) = \frac{1}{1 + e^{-(\mathbf{w}^T \mathbf{x} + b)}}
  $$

- **Probability of $ y = 0 $ given $ \mathbf{x} $**
  
  This can be computed as one minus the probability of $ y = 1 $:

  $$
  P(y=0|\mathbf{x}) = 1 - P(y=1|\mathbf{x}) = 1 - \frac{1}{1 + e^{-(\mathbf{w}^T \mathbf{x} + b)}}
  $$



## Exercise 1: Logistic regression model

### Fit the model

The code below
* Selects the ```LogisticRegression``` model
* Fits the model using all features and labels
* Calculates the accuracy on whole set and using cross-validation

Run the code.

In [ ]:
# Select the logistic regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

# Fit the model
model.fit(X,y)

# Calculate the accuracy on the whole set
print('Accuracy = ',round(model.score(X,y),2))

# Calculate cross-validated accuracy
accuracyCV(model,X,y)

### Task 1: Plot decision function

The **decision function** for univariate classifier is defined as $h=w_0+w_1x$. The fitted classifiers in scikit-learn return the values of the decision function using `model.decision_function`.

We will now plot the decision function of the fitted Logistic Regression model. Fist we will generate the feature space `x` for scaled EF, which covers interval [-2.5,2] as visible from the plot above.

Your task is to
* predict the decision function values `df` on the feature space `x`
* plot the decision function

The code for plotting the data is already there for you. Notices the output values of the decision function and the change of the scale in y-coordinate.

In [ ]:
# Generate feature space
x = np.linspace(-2.5,2,100).reshape(-1,1)
# Predict decision function
df = None
# Plot data
plotData(X,y,adjust_axis=False)
# Plot decision function
plt.plot(None,None,'k', label = 'Decision function')
plt.legend()

### Task 2: Plot decision boundary

The **decision boundary**  of a linear classifier is defined by the setting the decision function to zero. In univariate case it is $w_0+w_1x=0$.

Your task us to
* Find the intercept $w_0$ and slope $w_1$
* Calculate the decision boundary as $x=\frac{-w_0}{w_1}$.

*Hint:* Use `model.intercept_` and `model.coef_`. Remember that they are numpy arrays, so you need to select the first element

In [ ]:
# intercept is a 1D array
print('Intercept: ', np.around(model.intercept_,2))
# coeficients are a 2D array
print('Coefficients: ', np.around(model.coef_,2))

In [ ]:
# extract weights
w0 = None
w1 = None
print('w0 = ', round(w0,2))
print('w1 = ', round(w1,2))

In [ ]:
# calculate decision boundary - note it is a scalar
decision_boundary = None
print('Decision boundary is ', round(decision_boundary,2))

In [ ]:
# plot the data
plotData(X,y,adjust_axis=False)

# plot the decision boundary
plt.plot([decision_boundary,decision_boundary],[-0.1,1.1], 'k', label = 'Decision boundary')
_=plt.legend()

### Task 3: Plot probabilitic predictions

In binary logistic regression the confidence in predicted labels can be evaluated as $p(y=1|x)=\sigma(h(x))$ and $p(y=0|x)=1-\sigma(h(x))$. The probabilistic predictions are evaluated using function `model.predict_proba`.

Your task is to calculate probabilistic predictions `proba` for the whole feature space and plot them. Check the dimensions of `proba`, you will see that the function returns probabilities for both classes.

In [ ]:
# calculate probabilistic predictions
proba = None
# check the dimension
print('Dimensions of proba: ', proba.shape)

In [ ]:
# plot data
plotData(X,y)

# plot probabilistic predictions
plt.plot(x,proba[:,0],'b:', label = 'Healthy Probability')
plt.plot(x,proba[:,1],'r-', label = 'Heart Failure Probability')
plt.legend()

### Task 4: Diagnosis

A patient with suspicion of heart failure has an MRI scan. The ejection fraction is 40%. Decide whether the patient is likely to have a heart failure and calculate confidence in this decision.

**Answer:** None

In [ ]:
# EF=40%
x_new=40

# convert the feature to a 2D numpy array
# scale it using already fitted scaler
x_new = scaler.transform(np.array(x_new).reshape(-1,1))
print('Scaled EF: ', np.around(x_new,2))

# predict the label
y_new = None
print('Predicted label: ', y_new)

# predict confidence
proba_new = None
print('Predicted confidence: ', round(proba_new[0,y_new[0].astype(int)],2))